# Import usefull library

In [ ]:
import cv2
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer

# Load model

In [ ]:
HEIGHT = 480
WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2

In [ ]:
import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')

import efficientnet.tfkeras as efn

In [ ]:
weights_path = "../input/effnet-b5-model/EfficientNet_B5_Fullaug.h5"

In [ ]:
def get_model():
    base_model = efn.EfficientNetB5(include_top=False, weights= None , input_shape=(HEIGHT, WIDTH, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(top_dropout_rate)(x)
    outputs = Dense(CLASSES, activation='sigmoid')(x)
    model = Model(base_model.input, outputs) 
    return model

In [ ]:
model = get_model()
model.load_weights(weights_path)

# Preprocessing data test

In [ ]:
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission

In [ ]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(test_img+'/'+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def insect_augmentation(image, n_insects = 2, dark_insect=False, p = 0.5, insects_folder='../input/insect/insect'):
    aug_prob = random.random()
    flag = False
    if aug_prob < p:
        flag = True
        height, width, _ = image.shape  # target image width and height
        insects_images = os.listdir(insects_folder)
        img_shape = image.shape

        for _ in range(n_insects):
            insect = cv2.cvtColor(cv2.imread(os.path.join(insects_folder, random.choice(insects_images))), cv2.COLOR_RGB2BGR)
            insect = cv2.flip(insect, random.choice([-1, 0, 1]))
            insect = cv2.rotate(insect, random.choice([0, 1, 2]))
            insect = cv2.resize(insect, (width, height))

            h_height, h_width, _ = insect.shape  # insect image width and height
            roi_ho = random.randint(0, image.shape[0] - insect.shape[0])
            roi_wo = random.randint(0, image.shape[1] - insect.shape[1])
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(insect, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            #mask_inv = cv2.cvtColor(cv2.bitwise_not(mask),cv2.COLOR_BGR2GRAY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of insect in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_insect:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                insect_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                insect_fg = cv2.bitwise_and(insect, insect, mask=mask)

            # Put insect in ROI and modify the target image
            dst = cv2.add(img_bg, insect_fg, dtype = cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst


    return image

In [ ]:
def data_full_augment(image):
    
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    flag = False

    # Insect
    n = random.randint(1, 2)
    image = insect_augmentation(image, n_insects = n, dark_insect=False, p = 0.05)

    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
        flag = True
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
        flag = True
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        flag = True


    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        flag = True
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
        flag = True
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
        flag = True
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90
        flag = True
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
        flag = True
    elif p_crop > .4:
        HEIGHT1 = image.shape[0]
        WIDTH1 = image.shape[1]
        crop_size_h = tf.random.uniform([], int(HEIGHT1*.8), HEIGHT1, dtype=tf.float32)
        crop_size_w = tf.random.uniform([], int(WIDTH1*.8), WIDTH1, dtype=tf.float32)
        image = tf.image.random_crop(image, size=[crop_size_h, crop_size_w, 3])
        flag = True
    
    return image

# Prediction

In [ ]:
def process(img):
    img = data_full_augment(img)
    img = tf.cast(img, tf.float32) / 255.0
    img = np.array(img)
    return cv2.resize(img, (480, 480)).reshape(-1, 480, 480, 3)
def predict(img):
    img = load_image(img)
    tta_steps = 5
    predictions = []
    for i in range(tta_steps):
        pred = np.array(model.predict(process(img)))[0]
        predictions.append(pred)
    result = np.median(predictions, axis=0)
    
    return result

In [ ]:
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
label = []
for i in range(len(submission['image'])):
    test_images = submission['image'][i]
    preds = predict(test_images)
    answer = []
    for j in range(len(preds)):
        if preds[j] > 0.09:
            answer.append(n_label[j])
    if ("healthy" in answer) and (len(answer) > 1):
        answer.remove('healthy')
    if len(answer) == 0:
        answer.append('healthy')
    answer = ' '.join(answer)
    label.append(answer)
submission['labels'] = label
submission

# Submission

In [ ]:
submission.to_csv('submission.csv', index=False)